In [1]:
import json
import dateutil.parser

# custom Decoder
def DecodeDateTime(Dict):
    if 'publication_date' in Dict and Dict["publication_date"]!=None:
        Dict["publication_date"] = dateutil.parser.parse(Dict["publication_date"])
        return Dict
    else:
        return Dict

# use of object_hook
with open('cache_articles.json', 'r') as f:
    decoded_cache_articles = json.load(f,object_hook=DecodeDateTime)

In [2]:
import json
from Bio import Entrez


def related_keywords_mesh(papers):
    mesh_kewords_related = {}
    for paper in papers:
        if paper['mesh_terms']:
            mesh_kewords_related[paper['doi']] = paper['mesh_terms']
        else:
            mesh_kewords_related[paper['doi']] = paper['keywords']
    return mesh_kewords_related

In [5]:
import pandas as pd
File = 'train.csv'
df_train = pd.read_csv(File)
df_train = df_train[['Keywords (separated by ;)','doi']]
df_train

,Keywords (separated by ;),doi
0,affinity,10.1002/biot.201600357
1,affinity;intein,10.2174/1389203720666190208110416
2,affinity;Protein A,10.3390/ma9120994
3,affinity;Protein A;electrophoresis,10.1371/journal.pone.0139137
4,affinity;Protein A;Protein L,10.1080/19420862.2019.1662690
5,affinity;VHH,10.1128/aem.02595-14
6,bioconjugation;5MP,10.1021/jacs.7b00670
7,ELP,10.3389/fbioe.2019.00233
8,ELP,10.1016/j.jconrel.2014.06.028
9,ELP,10.1016/j.ab.2011.04.034


In [14]:
train_dataset = {}
for ind,row in df_train.iterrows():
    for key in row['Keywords (separated by ;)'].split(';'):
        if key in train_dataset:
            train_dataset[key].append(row['doi'])
        else:
            train_dataset[key] =[row['doi']]

In [16]:
train_papers = []
for key,value in train_dataset.items():
    if key in decoded_cache_articles:
        articles = decoded_cache_articles[key]
        for art in articles:
            if art['doi'] and (art['doi'] in value):
                train_papers.append(art)
len(train_papers)

66

In [17]:
%%time

with open('mesh_database.json','r') as f:
    mesh_database = json.load(f)
    
mesh_keywrds_extracted = related_keywords_mesh(train_papers)

len(mesh_keywrds_extracted)

Wall time: 1.14 s


18

In [19]:
reverse_train = {}
for key ,value in train_dataset.items():
    for val in value:
        if val in reverse_train:
            reverse_train[val].append(key)
        else:
            reverse_train[val] = [key]

In [20]:
reverse_train

{'10.1002/biot.201600357': ['affinity'],
 '10.2174/1389203720666190208110416': ['affinity', 'intein'],
 '10.3390/ma9120994': ['affinity', 'Protein A'],
 '10.1371/journal.pone.0139137': ['affinity', 'Protein A', 'electrophoresis'],
 '10.1080/19420862.2019.1662690': ['affinity', 'Protein A', 'Protein L'],
 '10.1128/aem.02595-14': ['affinity', 'VHH'],
 '10.1371/journal.pone.0025282': ['Protein A'],
 '10.1080/19420862.2019.1565749': ['Protein A'],
 '10.1038/s41467-018-05403-1': ['Protein L', 'membrane'],
 '10.1021/jacs.7b00670': ['bioconjugation', '5MP'],
 '10.3389/fbioe.2019.00233': ['ELP'],
 '10.1016/j.jconrel.2014.06.028': ['ELP'],
 '10.1016/j.ab.2011.04.034': ['ELP'],
 '10.1002/0471140864.ps0611s61': ['ELP'],
 '10.1016/j.jconrel.2015.11.010': ['ELP'],
 '10.1021/bm400167h': ['ELP'],
 '10.1073/pnas.2000223117': ['membrane'],
 '10.1016/j.jmb.2018.06.038': ['membrane']}

In [21]:
ranking_schema = {}
for key ,value in mesh_keywrds_extracted.items():
    values = [val[0] for val in value]
    for keywrd in reverse_train[key]:
        if keywrd in ranking_schema:
            ranking_schema[keywrd].extend(values)
        else:
            ranking_schema[keywrd] = values
    ranking_schema[keywrd] = list(set(ranking_schema[keywrd]))

In [23]:
ranking_schema

{'affinity': ['Exotoxins',
  'Antibodies, Monoclonal',
  'Bacterial Toxins',
  'Chromatography, Affinity',
  'Immunoglobulin G',
  'Prothrombin',
  'Antibodies',
  'Staphylococcal Protein A',
  'Humans',
  'Binding Sites',
  'Biocatalysis',
  'Biosensing Techniques',
  'Biotechnology',
  'Inteins',
  'Peptides',
  'Protein Binding',
  'Protein Conformation',
  'Protein Engineering',
  'Protein Splicing',
  'Proteins',
  'Recombinant Proteins',
  'F',
  'P',
  'P',
  'P',
  'P',
  'a',
  'a',
  't',
  'Bacteria',
  'Chromatography, Affinity',
  'Fermentation',
  'Metabolic Engineering',
  'Polyesters',
  'Protein Binding',
  'Recombinant Proteins',
  'Amino Acid Sequence',
  'Biotin',
  'Biotinylation',
  'Carrier Proteins',
  'Chromatography, Affinity',
  'Cysteine',
  'Electrophoresis, Polyacrylamide Gel',
  'Kinetics',
  'Ligands',
  'Maltose-Binding Proteins',
  'Models, Molecular',
  'Molecular Sequence Data',
  'Peptides',
  'Protein Binding',
  'Protein Engineering',
  'Recombina

In [25]:
import json
with open('train_dataset_mesh.json','w') as f:
    json.dump(ranking_schema,f)